## 把 sft_training_data_final.jsonl 拆成測試跟驗證集的地方

In [1]:
# 安裝必要的函式庫
# 注意：安裝完成後，若遇到 "requires the latest version of bitsandbytes" 錯誤，
# 請務必「重啟執行階段」(Runtime > Restart session)，然後跳過此 cell 直接執行後續程式碼。
!pip install -U -q bitsandbytes transformers accelerate
print("套件安裝完成。")
print("⚠️以此 cell 安裝完畢後，強烈建議點擊上方選單 'Runtime' > 'Restart session' 重啟環境，以確保 CUDA 函式庫正確載入。")

套件安裝完成。
⚠️以此 cell 安裝完畢後，強烈建議點擊上方選單 'Runtime' > 'Restart session' 重啟環境，以確保 CUDA 函式庫正確載入。


In [2]:
import json
import os
import re
import time
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import sys
import subprocess

# 自動檢查並安裝必要的 Hugging Face 套件，並進行環境驗證
def install_and_validate_dependencies():
    print("=== 環境檢查與套件安裝 ===")

    # 1. 檢查 GPU
    if not torch.cuda.is_available():
        print("❌ 警告：未偵測到 GPU！模型推論將極其緩慢或無法執行。")
        print("   請至選單 'Runtime' > 'Change runtime type' 選擇 T4 GPU。")
    else:
        gpu_name = torch.cuda.get_device_name(0)
        vram = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"✅ 偵測到 GPU: {gpu_name} (VRAM: {vram:.2f} GB)")

        # 簡單檢查 VRAM 是否足夠 (7B 4bit 需要約 6GB)
        if vram < 6:
            print("⚠️ 警告：GPU VRAM 可能不足以執行 7B 模型，可能會發生 OOM 錯誤。")

    # 2. 驗證套件版本與功能 (不在此處重複安裝，以免版本衝突，請依賴上方的 !pip install)
    try:
        import transformers
        import accelerate
        import bitsandbytes
        print(f"✅ Transformers version: {transformers.__version__}")
        print(f"✅ Accelerate version: {accelerate.__version__}")
        print(f"✅ Bitsandbytes version: {bitsandbytes.__version__}")
    except ImportError as e:
        print(f"❌ 套件載入失敗: {e}")
        print("請確認已執行上方的 pip install 指令，並建議重啟 Runtime。")
        return False

    return True

if not install_and_validate_dependencies():
    sys.exit("環境驗證失敗，請檢查錯誤訊息。")

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

=== 環境檢查與套件安裝 ===
✅ 偵測到 GPU: Tesla T4 (VRAM: 14.74 GB)
✅ Transformers version: 4.57.3
✅ Accelerate version: 1.12.0
✅ Bitsandbytes version: 0.49.0


In [3]:
# manually upload 'sft_training_data_final.jsonl'

In [4]:
# ==========================================
# 設定區
# ==========================================
INPUT_FILE = 'sft_training_data_final.jsonl'
OUTPUT_DIR = 'split_dataset_smart'
RANDOM_SEED = 42

# 目標切分數量
TRAIN_SIZE = 300
VAL_SIZE = 29
TEST_SIZE = 40

# 使用 Hugging Face 模型 (免費方案: 本地運行)
# 這裡選用 Qwen2.5-7B-Instruct，中文能力強且可透過 4-bit 量化跑在 Colab T4 上
HF_MODEL_ID = "Qwen/Qwen2.5-7B-Instruct"

# 定義職災類別標籤 (讓 LLM 從中選擇)
INCIDENT_CATEGORIES = [
    "墜落、滾落",       #
    "跌倒",             #
    "衝撞",             # (指人體以自身動力碰撞物體)
    "物體飛落",         #
    "物體倒塌、崩塌",   #
    "被撞",             # (指人體被飛來、落下、滑動之物體擊中)
    "被夾、被捲",       #
    "被切、割、擦傷",   #
    "踩踏",             #
    "溺斃",             #
    "與高溫、低溫接觸", #
    "與有害物等之接觸", # (含吸入中毒)
    "感電",             #
    "爆炸",             #
    "物體破裂",         #
    "火災",             #
    "不當動作",         #
    "交通事故",         # 合併 (公路、鐵路、船舶等)
    "其他"              # (無法歸類)
]

In [5]:
def clean_text_for_preview(text):
    """
    使用 Regex 強力清洗：
    1. 去除所有非中文字符 (去除標點、數字、英文)
    2. 去除 '○' (去識別化符號)
    3. 去除常見無意義詞
    """
    # 僅保留中文字 (\u4e00-\u9fa5)
    text = re.sub(r'[^\u4e00-\u9fa5]', '', text)
    # 去除特定雜訊字
    text = re.sub(r'[○oO]', '', text)
    return text

In [6]:
def get_category_from_local_llm(text, model, tokenizer):
    """
    使用本地 Hugging Face 模型進行分類。
    """
    snippet = text[:600]

    system_prompt = """
    你是一位精通台灣職業安全法規的專家。請依據「職業災害類型分類表」對事故進行分類。

    【重要原則】
    1. 請區分「媒介物」與「災害類型」。例如：被「堆高機」撞到，災害類型應選「被撞」，而非其他。
    2. 「衝撞」指人去撞物體；「被撞」指物體來撞人。
    3. 若涉及墜落，優先選擇「墜落、滾落」。
    4. 若涉及觸電，選擇「感電」。
    """

    user_prompt = f"""
    請閱讀以下職災事故描述，並從下列標準清單中，選出最主要的一個「災害類型」：

    {json.dumps(INCIDENT_CATEGORIES, ensure_ascii=False)}

    事故描述：
    {snippet}

    請直接輸出類別名稱（例如：被夾、被捲），不要輸出任何解釋或其他文字。
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    # 使用 Chat Template 格式化輸入
    text_input = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text_input], return_tensors="pt").to(model.device)

    try:
        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=20,     # 不需要太長，只要類別名稱
            temperature=0.1,       # 低溫確保穩定
            do_sample=False        # 關閉採樣以獲得確定性結果
        )

        # 取出新增的 tokens (移除 input 部分)
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        return response.strip()
    except Exception as e:
        print(f"Inference Error: {e}")
        return "其他"

In [7]:
def main():
    # 1. 讀取資料
    print(f"正在讀取 {INPUT_FILE}...")
    data = []
    if not os.path.exists(INPUT_FILE):
        print(f"錯誤：找不到檔案 {INPUT_FILE}。請先執行上傳步驟。")
        return

    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))

    df = pd.DataFrame(data)

    # 2. 準備模型 (Loading Local Model)
    cache_file = "incident_categories_cache.json"
    categories = []

    if os.path.exists(cache_file):
        print("偵測到分類快取檔，正在載入... (跳過模型載入)")
        with open(cache_file, 'r', encoding='utf-8') as f:
            categories = json.load(f)
    else:
        print(f"正在載入模型 {HF_MODEL_ID} (4-bit 量化)...")

        # 設定 4-bit 量化配置
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16
        )

        try:
            tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_ID)
            model = AutoModelForCausalLM.from_pretrained(
                HF_MODEL_ID,
                quantization_config=quantization_config,
                device_map="auto",
                trust_remote_code=True
            )
        except Exception as e:
            print(f"❌ 模型載入失敗: {e}")
            print("請確認已安裝 accelerate 和 bitsandbytes，並使用 GPU 環境。")
            return

        # === 模型功能驗證 (Dry Run) ===
        print("\n=== 正在驗證模型功能 (Dry Run) ===")
        try:
            test_prompt = "測試"
            test_input = tokenizer(test_prompt, return_tensors="pt").to(model.device)
            _ = model.generate(**test_input, max_new_tokens=5, do_sample=False)
            print("✅ 模型載入成功且推論功能正常！開始處理資料...\n")
        except Exception as e:
            print(f"❌ 模型功能驗證失敗: {e}")
            print("這可能是由於 VRAM 不足或 CUDA 配置錯誤。")
            return
        # =============================

        print(f"開始對 {len(df)} 筆資料進行分類...")
        for i, row in df.iterrows():
            if i % 10 == 0: print(f"進度: {i}/{len(df)}")

            cat = get_category_from_local_llm(row['input'], model, tokenizer)

            # 清洗回傳結果
            found_cat = "其他"
            for valid_cat in INCIDENT_CATEGORIES:
                if valid_cat in cat:
                    found_cat = valid_cat
                    break
            categories.append(found_cat)

        # 存檔
        with open(cache_file, 'w', encoding='utf-8') as f:
            json.dump(categories, f, ensure_ascii=False)

    df['category'] = categories

    # 3. 顯示分類結果統計
    print("\n[AI 判斷] 職災類型分佈：")
    print(df['category'].value_counts())

    # 4. 分層切分 (Stratified Split)
    print("\n正在進行分層切分...")

    try:
        df_train, df_temp = train_test_split(
            df,
            train_size=TRAIN_SIZE,
            stratify=df['category'],
            random_state=RANDOM_SEED
        )
    except ValueError as e:
        print(f"警告：某些類別樣本太少，無法完美分層。轉為隨機切分。(錯誤: {e})")
        df_train, df_temp = train_test_split(df, train_size=TRAIN_SIZE, random_state=RANDOM_SEED)

    try:
        df_test, df_val = train_test_split(
            df_temp,
            train_size=TEST_SIZE,
            stratify=df_temp['category'],
            random_state=RANDOM_SEED
        )
    except ValueError:
        print("警告：剩餘資料類別分佈稀疏，第二階段轉為隨機切分。")
        df_test, df_val = train_test_split(
            df_temp,
            train_size=TEST_SIZE,
            random_state=RANDOM_SEED
        )

    # 5. 輸出結果與儲存
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)

    def save_split(df_split, name):
        path = os.path.join(OUTPUT_DIR, name)
        records = df_split.drop(columns=['category']).to_dict('records')
        with open(path, 'w', encoding='utf-8') as f:
            for r in records:
                f.write(json.dumps(r, ensure_ascii=False) + '\n')

        print(f"\n檔案: {name} ({len(records)} 筆)")
        print("分佈概況:", df_split['category'].value_counts().to_dict())

    save_split(df_train, 'train.jsonl')
    save_split(df_val, 'val.jsonl')
    save_split(df_test, 'test.jsonl')

    print(f"\n全部完成！檔案已儲存於 {OUTPUT_DIR}/")

In [8]:
if __name__ == "__main__":
    main()

正在讀取 sft_training_data_final.jsonl...
正在載入模型 Qwen/Qwen2.5-7B-Instruct (4-bit 量化)...


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



=== 正在驗證模型功能 (Dry Run) ===


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


✅ 模型載入成功且推論功能正常！開始處理資料...

開始對 369 筆資料進行分類...
進度: 0/369
進度: 10/369
進度: 20/369
進度: 30/369
進度: 40/369
進度: 50/369
進度: 60/369
進度: 70/369
進度: 80/369
進度: 90/369
進度: 100/369
進度: 110/369
進度: 120/369
進度: 130/369
進度: 140/369
進度: 150/369
進度: 160/369
進度: 170/369
進度: 180/369
進度: 190/369
進度: 200/369
進度: 210/369
進度: 220/369
進度: 230/369
進度: 240/369
進度: 250/369
進度: 260/369
進度: 270/369
進度: 280/369
進度: 290/369
進度: 300/369
進度: 310/369
進度: 320/369
進度: 330/369
進度: 340/369
進度: 350/369
進度: 360/369

[AI 判斷] 職災類型分佈：
category
墜落、滾落       218
被撞           52
被夾、被捲        27
跌倒           22
感電           16
物體倒塌、崩塌      10
其他            5
物體飛落          4
火災            4
與有害物等之接觸      3
與高溫、低溫接觸      2
溺斃            2
交通事故          2
不當動作          1
爆炸            1
Name: count, dtype: int64

正在進行分層切分...
警告：某些類別樣本太少，無法完美分層。轉為隨機切分。(錯誤: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.)
警告：剩餘資料類別分佈稀疏，第二階段轉為隨機切分。

檔案: train.jsonl (300 筆)